In [1]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
#splitedRdd = sc.parallelize(splitedRdd.take(10000))
splitedRddSample1 = sc.parallelize(splitedRdd.takeSample(True, 2000,0))
splitedRddSample2 = sc.parallelize(splitedRdd.takeSample(True, 2000,1))
splitedRddSample1.take(1)
#(uid,[[train],[test]]

KeyboardInterrupt: 

In [ ]:
numRecList = 5
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList     
def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores
def evaluation_rr1(testItem, recList):
    scores = 0
    for i in range(len(recList)):
        if testItem == recList[i]:
            scores = 1.0 / (i+1)
            break
    return scores
def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0
                    num = num + 1  
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def evaluation_precision(testItem, recList):
    scores = 0
    if testItem in recList:
        scores = 1.0
    return scores  


def print_result(finalScore, algorithm_name):
    f = open('result/mfu_test_result3.txt','a')
    f.write('2000 users with duplicated evaluations with installed data')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()

In [ ]:
from operator import add
from collections import Counter
f = open('result/mfu_test_result2.txt','a')
f.write('-------normal mfu-----')
f.write('\n')
f.close()
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    
    trainList = listGroup[0]     #0.8 train set
    testList = listGroup[1]    #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
#        testcount = Counter(testList_items).most_common()
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        RecommenderDuplicate1 =  [t[0] for t in trainList]        #take only id for train set
        Recommender1 = remove_duplicates(RecommenderDuplicate1)    #remove duplicate
        Recommender = [[t[0],t[1]*5] if t[0] in Recommender1[:10] else t for t in Recommender]
#         Recommender = [[t[0],1.0*t[1]/numTrain] for t in Recommender]
#         testcount = [[t[0],1.0*t[1]/numTest] for t in testcount]
       
    #weighted
#         Recommender =  [[t,0] for t in Recommender] 
#         for i in range(len(Recommender)):
#             for j in range(len(trainList)):
#                 if trainList[j][0] == Recommender[i][0]:
#                     if len(trainList)>0 and (trainList[-1][1][0] - trainList[j][1][0]) == 0:
#                         weight = 1
#                     elif len(trainList)>0:
#                         weight = (trainList[-1][1][0] - trainList[j][1][0])/86400
#                     else:
#                         weight = 0
#                     if weight != 0:
#                         Recommender[i][1] = Recommender[i][1] + 1.0/weight
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRdd1.map(mfuFunction)
print_result(finalScore,'mfu_sample1')
finalScore = splitedRdd2.map(mfuFunction)
print_result(finalScore,'mfu_sample2')

In [ ]:
from operator import add
from collections import Counter
f = open('result/mfu_test_result2.txt','a')
f.write('-------normal mfu-----')
f.write('\n')
f.close()
def mfuFunctionInstalled(line):
    listGroup = line[1]
    l = len(listGroup)  
    
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    
    trainList = listGroup[0]     #0.8 train set
    testList = listGroup[1]    #0.2 test set
    installedList = listGroup[2]
    numTrain = len(trainList)
    numTest = len(testList)
    
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
#        testcount = Counter(testList_items).most_common()
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        RecommenderDuplicate1 =  [t[0] for t in trainList]        #take only id for train set
        Recommender1 = remove_duplicates(RecommenderDuplicate1)    #remove duplicate
        Recommender = [[t[0],t[1]*5] if t[0] in Recommender1[:10] else t for t in Recommender]
#         Recommender = [[t[0],1.0*t[1]/numTrain] for t in Recommender]
#         testcount = [[t[0],1.0*t[1]/numTest] for t in testcount]
       
    #weighted
#         Recommender =  [[t,0] for t in Recommender] 
#         for i in range(len(Recommender)):
#             for j in range(len(trainList)):
#                 if trainList[j][0] == Recommender[i][0]:
#                     if len(trainList)>0 and (trainList[-1][1][0] - trainList[j][1][0]) == 0:
#                         weight = 1
#                     elif len(trainList)>0:
#                         weight = (trainList[-1][1][0] - trainList[j][1][0])/86400
#                     else:
#                         weight = 0
#                     if weight != 0:
#                         Recommender[i][1] = Recommender[i][1] + 1.0/weight
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRdd1.map(mfuFunctionInstalled)
print_result(finalScore,'mfu_sample1')
finalScore = splitedRdd2.map(mfuFunctionInstalled)
print_result(finalScore,'mfu_sample2')